# Transformer - Probabilistic Forecast

## 0. imports

In [1]:
%load_ext jupyter_black

In [2]:
import sys

sys.path.append("..")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import namedtuple

In [4]:
from src.dataset import ETTDataModule
from src.model import Transformer

## 1. prev setting

In [5]:
dm_params = {
    "data_path": "../data/ETT-small/ETTh1.csv",
    "task": "M",
    "freq": "h",
    "target": "OT",
    "seq_len": 96,
    "label_len": 48,
    "pred_len": 96,
    "use_scaler": True,
    "use_time_enc": True,
    "batch_size": 32,
}


dm = ETTDataModule(**dm_params)

In [6]:
# transformer_params
Config = namedtuple(
    "Config",
    [
        "task_name",
        "pred_len",
        "seq_len",
        "num_class",
        "enc_in",
        "dec_in",
        "c_out",
        "d_model",
        "embed_type",
        "freq",
        "dropout",
        "n_heads",
        "d_keys",
        "d_values",
        "d_ff",
        "scale",
        "attention_dropout",
        "output_attention",
        "activation",
        "num_enc_layers",
        "num_dec_layers",
    ],
)

configs = Config(
    task_name="long_term_forecast",
    pred_len=96,
    seq_len=None,
    num_class=None,
    enc_in=7,
    dec_in=7,
    c_out=7,
    d_model=512,
    embed_type="time_features",
    freq="h",
    dropout=0.1,
    n_heads=8,
    d_keys=None,
    d_values=None,
    d_ff=2048,
    scale=None,
    attention_dropout=0.1,
    output_attention=True,
    activation="gelu",
    num_enc_layers=2,
    num_dec_layers=1,
)

In [7]:
model = Transformer(**configs._asdict())

In [8]:
train_dataloader = dm.train_dataloader()
batch = next(iter(train_dataloader))

# decoder input
label_len = 48
dec_inp = torch.zeros_like(batch["future_values"][:, -configs.pred_len :, :]).float()
dec_inp = torch.cat([batch["future_values"][:, :label_len, :], dec_inp], dim=1).float()

In [9]:
output = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    future_values=dec_inp,
    future_time_features=batch["future_time_features"],
)

In [10]:
output["last_hidden_states"]

tensor([[[-0.7532,  0.5016, -0.6780,  ..., -0.9313, -0.7630,  0.2696],
         [-0.7095,  1.0485, -0.3392,  ..., -0.5668, -1.0276,  0.3088],
         [-0.6347,  0.9774, -0.4741,  ..., -0.8312, -1.0052,  0.6853],
         ...,
         [-0.7334,  0.1655,  0.2394,  ..., -0.2001, -0.0030,  0.6181],
         [-0.5439,  0.6911,  0.4149,  ..., -0.2567,  0.3115,  0.1636],
         [-0.6202,  1.2147,  0.3297,  ..., -0.2797, -0.2696,  0.2670]],

        [[ 0.1508, -0.3699, -1.1867,  ...,  0.1533,  0.1827, -0.7236],
         [-0.0193, -0.2874, -0.8082,  ..., -0.2953,  0.2752, -0.6535],
         [-0.1565, -0.5074, -0.8264,  ..., -0.1849,  0.0810, -0.1247],
         ...,
         [-0.4047, -0.2092,  0.2076,  ..., -0.0950,  0.4518,  0.2984],
         [-0.0614, -0.0345,  0.2961,  ..., -0.2699,  0.5922,  0.1125],
         [ 0.0262, -0.5969, -0.6184,  ..., -0.6735,  0.8110, -0.0223]],

        [[ 0.6263, -0.0202,  0.3586,  ..., -0.4256,  0.2879,  0.0959],
         [ 0.6704,  0.1471,  0.8849,  ..., -0

In [11]:
batch["future_values"].shape

torch.Size([32, 144, 7])